In [29]:
# This is for my own setup

!ls -la /content
!git clone https://github.com/mehrdad-mirpourian/nlu_hw2.git
!ls -la /content/nlu_hw2


import os
os.chdir("/content/nlu_hw2")
print("Current directory:", os.getcwd())  # Should print /content/nlu_hw2

!ls -la

total 20
drwxr-xr-x 1 root root 4096 Mar  9 13:38 .
drwxr-xr-x 1 root root 4096 Mar  9 13:37 ..
drwxr-xr-x 4 root root 4096 Mar  6 14:29 .config
drwxr-xr-x 5 root root 4096 Mar  9 14:46 nlu_hw2
drwxr-xr-x 1 root root 4096 Mar  6 14:29 sample_data
fatal: destination path 'nlu_hw2' already exists and is not an empty directory.
total 240
drwxr-xr-x 5 root root   4096 Mar  9 14:46 .
drwxr-xr-x 1 root root   4096 Mar  9 13:38 ..
drwxr-xr-x 8 root root   4096 Mar  9 13:38 .git
-rw-r--r-- 1 root root 124125 Mar  9 13:38 hw2-pset.ipynb
-rw-r--r-- 1 root root  79299 Mar  9 13:38 My_Checks.ipynb
drwxr-xr-x 3 root root   4096 Mar  9 14:46 nlu_hw2
drwxr-xr-x 2 root root   4096 Mar  9 13:39 __pycache__
-rw-r--r-- 1 root root   4903 Mar  9 13:38 README.md
-rw-r--r-- 1 root root   1310 Mar  9 13:38 test_model.py
-rw-r--r-- 1 root root   3952 Mar  9 13:38 train_model.py
Current directory: /content/nlu_hw2
total 240
drwxr-xr-x 5 root root   4096 Mar  9 14:46 .
drwxr-xr-x 1 root root   4096 Mar  9 13:38

In [30]:
! pip install datasets evaluate optuna --quiet # install datasets if it is not included in your environment

In [31]:
import torch
from collections.abc import Iterable
from datasets import load_dataset, Dataset

# Model and tokenizer from 🤗 Transformers
from transformers import AutoModelForSequenceClassification, \
    BertForSequenceClassification, BertTokenizerFast


In [32]:
# Mehi: When I start running the model I need to check this part and run it
# Code you will write for this assignment
from train_model import init_model, preprocess_dataset, init_trainer
from test_model import init_tester

In [33]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
model = BertForSequenceClassification.from_pretrained(
    "prajjwal1/bert-tiny", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
# This code does exactly the same thing as the previous code cell
model = AutoModelForSequenceClassification.from_pretrained(
    "prajjwal1/bert-tiny", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
tokenizer = BertTokenizerFast.from_pretrained("prajjwal1/bert-tiny")

In [37]:
# Because 🤗 Transformers supports multiple deep learning libraries, you will
# need to use the keyword parameter return_tensors in order to indicate that
# you want your inputs to be returned in PyTorch format.
inputs = tokenizer(["Hello world!", "How are you?"], padding=True,
                   return_tensors="pt")
inputs

{'input_ids': tensor([[ 101, 7592, 2088,  999,  102,    0],
        [ 101, 2129, 2024, 2017, 1029,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1]])}

In [38]:
model.eval()
with torch.no_grad():
    outputs = model(**inputs)

print(outputs, end="\n\n")

# Use the dot operator to access parts of the output
print(outputs.logits)

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.0942, -0.2167],
        [ 0.2305, -0.1877]]), hidden_states=None, attentions=None)

tensor([[ 0.0942, -0.2167],
        [ 0.2305, -0.1877]])


## **Problem 1d. Prepare Dataset (Code, 10 Points)**

In [39]:
# Load IMDb dataset and create validation split
imdb = load_dataset("imdb")
split = imdb["train"].train_test_split(.2, seed=3463)
imdb["train"] = split["train"]
imdb["val"] = split["test"]
del imdb["unsupervised"]

In [41]:
def preprocess_dataset(dataset: Dataset, tokenizer: BertTokenizerFast) \
        -> Dataset:
    """
    Problem 1d: Implement this function.

    Preprocesses a dataset using a Hugging Face Tokenizer and prepares
    it for use in a Hugging Face Trainer.

    :param dataset: A dataset
    :param tokenizer: A tokenizer
    :return: The dataset, prepreprocessed using the tokenizer
    """

    def tokenize_function(examples):
        """
        Tokenizes input text using Hugging Face BERT tokenizer.
        Handles multiple edge cases to ensure robust preprocessing.
        """

        # Edge Case: Handle missing or empty text
        if "text" not in examples or examples["text"] is None:
            examples["text"] = ["[EMPTY]"] * len(examples.get("label", [0] * len(examples)))

        # Edge Case: Handle empty strings or unexpected formats
        examples["text"] = [t if isinstance(t, str) and t.strip() else "[EMPTY]" for t in examples["text"]]

        # Edge Case: Handle different data structures (alternative keys)
        if not isinstance(examples["text"], list):
            examples["text"] = [str(examples["text"])]

        try:
            tokenized = tokenizer(
                examples["text"],
                padding="max_length",  # Ensures all sequences are of equal length
                truncation=True,       # Ensures long sequences are truncated
                max_length=512        # As per Appendix A.2 of BERT paper
            )
        except Exception as e:
            # Edge Case: Handle tokenization failures
            print(f"Tokenization error: {e}")
            return {
                "input_ids": [[0] * 512],  # Default empty tokens
                "token_type_ids": [[0] * 512],
                "attention_mask": [[0] * 512]
            }

        return tokenized

    # Edge Case: Handle large batch sizes by limiting batch processing
    tokenized_dataset = dataset.map(tokenize_function, batched=True, batch_size=32)

    return tokenized_dataset




In [42]:
imdb["train"] = preprocess_dataset(imdb["train"], tokenizer)
imdb["val"] = preprocess_dataset(imdb["val"], tokenizer)
imdb["test"] = preprocess_dataset(imdb["test"], tokenizer)

# Visualize the preprocessed dataset
for k, v in imdb["val"][:2].items():
    print("{}:\n{}\n{}\n".format(k, type(v),
                                 [item[:20] if isinstance(item, Iterable) else
                                 item for item in v[:5]]))

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

text:
<class 'list'>
['As so many others ha', 'When converting a bo']

label:
<class 'list'>
[1, 0]

input_ids:
<class 'list'>
[[101, 2004, 2061, 2116, 2500, 2031, 2517, 1010, 2023, 2003, 1037, 6919, 4516, 1012, 2182, 2003, 1037, 2862, 1997, 1996], [101, 2043, 16401, 1037, 2338, 2000, 2143, 1010, 2009, 2003, 3227, 1037, 2204, 2801, 2000, 2562, 2012, 2560, 2070, 1997]]

token_type_ids:
<class 'list'>
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

attention_mask:
<class 'list'>
[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]

